In [2]:
import cv2 as cv
def extract_frames(videoPath, framePath):
    cap = cv.VideoCapture(videoPath)
    i=0
    while(cap.isOpened()):
        ret, frame = cap.read()
        if (ret == False):
            break
        i += 1
        frame = cv.resize(frame, (768,432))
        cv.imwrite(framePath + "/" + str(i) + ".jpg", frame)
    cap.release()
    cv.destroyAllWindows()

In [15]:
extract_frames("zeynep.mp4", "C:/Users/Zeynep/Desktop/SVMfaceR/dataset/zeynep")

In [14]:
extract_frames("serkan.mp4", "C:/Users/Zeynep/Desktop/SVMfaceR/dataset/serkan")

In [3]:
from deepface import DeepFace 
model = DeepFace.build_model("Facenet512")

In [5]:
import numpy as np
ssd_face_model = cv.dnn.readNetFromCaffe('deploy.prototxt.txt', 'res10_300x300_ssd_iter_140000.caffemodel')
path = "C:/Users/Zeynep/Desktop/SVMfaceR/dataset/"
def get_face_ssdnet(frame, image_size = 224):
	(h, w) = frame.shape[:2]
	resized_image = cv.resize(frame, (300, 300))
	blob = cv.dnn.blobFromImage(resized_image, 1.0, (300, 300), (104.0, 177.0, 123.0))
	#up here returns a blob which is our input image after mean subtraction, normalizing, and channel swapping.
	ssd_face_model.setInput(blob)
	detections = ssd_face_model.forward()
	box = detections[0, 0, 0, 3:7] * np.array([w, h, w, h])
	(startX, startY, endX, endY) = box.astype("int32")
	#cv.rectangle(frame, (startX, startY), (endX, endY), (150, 0, 255), 3)
	frame = frame[startY:endY, startX:endX]
	return frame


In [6]:
import os 
def save_detected_faces(main_path, save_path):
    img_names = os.listdir(main_path)
    rand_indexes = np.random.choice(range(len(img_names)), 100, replace=False)
    for j in rand_indexes:
        img = img_names[j]
        frame = cv.imread(os.path.join(main_path, img))
        cv.imwrite(save_path + 's' + img, get_face_ssdnet(frame))

In [13]:
save_detected_faces(main_path = "C:/Users/Zeynep/Desktop/SVMfaceR/dataset/serkan", save_path = "C:/Users/Zeynep/Desktop/SVMfaceR/dataset/detectedserkan/")

In [11]:
save_detected_faces(main_path = "C:/Users/Zeynep/Desktop/SVMfaceR/dataset/zeynep", save_path = "C:/Users/Zeynep/Desktop/SVMfaceR/dataset/detectedzeynep/")

In [8]:
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(160, 160))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = preprocess_input(img)
    return img

In [9]:
import time
dataset_path = "C:/Users/Zeynep/Desktop/SVMfaceR/dataset/detectedfaces"
img_names = os.listdir(dataset_path)
labels = []
distances = []
count = 0
for i in range(len(img_names)):
    for j in range(len(img_names)):
        if(count % 100 == 0):
            print(count)
        im1 = img_names[i]
        im2 = img_names[j]

        if im1[0] == im2[0]:
            labels.append(1)
        else:
            labels.append(0)

        abs_path_im1 = os.path.join(dataset_path, im1)
        abs_path_im2 = os.path.join(dataset_path, im2)

        img1_representation = model.predict(preprocess_image(abs_path_im1))[0,:]
        img2_representation = model.predict(preprocess_image(abs_path_im2))[0,:]

        distances.append(np.abs(img1_representation - img2_representation))
        count += 1

        
        

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [10]:
distances = np.array(distances)
labels = np.array(labels)

In [11]:
np.save('distances.npy', distances)
np.save('labels.npy', labels)

In [5]:
import numpy as np
distances = np.load('distances.npy')
labels = np.load('labels.npy')

In [6]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train, y_test = train_test_split(distances, labels, random_state=47, shuffle = True)

In [8]:
from sklearn.svm import SVC
import warnings
from sklearn.model_selection import GridSearchCV
k_fold = 3
warnings.filterwarnings("ignore")
scores = ["precision_macro","recall_macro", "accuracy", "f1_weighted"]
param_grid = {'C': [0.1,1, 10], 
              'gamma': [1,0.1,0.01],}

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(kernel= 'rbf'), param_grid, scoring="%s" % score,
                       cv = k_fold, refit=True,verbose=2)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_["mean_test_score"]
    stds = clf.cv_results_["std_test_score"]
    for mean, std, params in zip(means, stds, clf.cv_results_["params"]):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    print()

# Tuning hyper-parameters for precision_macro

Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END .....................................C=0.1, gamma=1; total time= 4.3min
[CV] END .....................................C=0.1, gamma=1; total time= 4.2min
[CV] END .....................................C=0.1, gamma=1; total time= 4.3min
[CV] END ...................................C=0.1, gamma=0.1; total time= 3.7min
[CV] END ...................................C=0.1, gamma=0.1; total time= 3.7min
[CV] END ...................................C=0.1, gamma=0.1; total time= 3.7min
[CV] END ..................................C=0.1, gamma=0.01; total time= 3.2min
[CV] END ..................................C=0.1, gamma=0.01; total time= 3.3min
[CV] END ..................................C=0.1, gamma=0.01; total time= 3.0min
[CV] END .......................................C=1, gamma=1; total time= 5.7min
[CV] END .......................................C=1, gamma=1; total time= 7.2min
[C

In [7]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train, y_train)

SVC()

In [8]:
model.score(X_test, y_test)

0.9951

In [9]:
model.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [10]:
import pickle
pkl_filename = "pickle_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model, file)